# USA Airlines analysis By [BTS](https://transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time) 

### Introduction

The United States Department of Transportation has Flight Stats available through the Bureau of Transportation Statistics.
This data comes from The Bureau of Transportation Statistics and tracks destinations, distance, and delay information of flights across U.S. For this project, I chose the years **1995** through **February 2020** 

### Notes

Throughout this notebook, you'll notice that most operations are single cell, this was because my machine kept running out of usable memory. The dataset was really large and required as much memory as possible for each operation. You will see multiple cells of code that could possibly be more code efficient, but were lengthened to reduce memory errors. You will also notice that after certain blocks of code, I outputted the dataframe to a csv. Some blocks of code took more than an hour to run. So to save my progress and make sure that I would not waste time re-running previous code, I would output my progress to a csv and then read it back in. I did not include these datasets in the repository, but have kept the code in the notebook.

The BTS website did not allow users to download full years of data. I downloaded data from the BTS website month by month from **1995 to February 2020*** and then used pandas to merge the data together.after cleaning each Csv  file then save it back to another csv to reduce memory usage and drop unnessasry columns in this **Gigantic** dataset.

# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `2019_1.csv` into a dataframe and inspect the first few lines. this CSV file contains data for all airports for the period 1-1-2019 till 31-1-2019


In [49]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import math
import datetime
import time as t

In [50]:
# load january 2019 dataset
df1=pd.read_csv(r"D:\on time data for airlines\2019\2019_1.csv",low_memory=False)

In [51]:
df1.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583985 entries, 0 to 583984
Columns: 110 entries, Year to Unnamed: 109
dtypes: float64(69), int64(21), object(20)
memory usage: 490.1+ MB


#### Memory usage_1
Due to high data load this file only which contain just one month data uses **490 MB** so let's munge our data and keep necessary columns only to decrease or eliminate this issue

## Memory size issue

it's clear that after we have run this single month data for 2019 we use about **500 MB** of memory so what about data soan from 2020 to 1995 so about 25 years of data each year has 12 csv file so let's overcome this problem.

### Soluntions

1- Fisrt thing i will drop all unrelevant columns or columns provide unuseful information for my analysis 

2- I will invastigate all columns to find ranges of numerical data then i will downcast it to proper data type

### Step 1 : remove Unnecessary columns

### Un-necessary Data for our analysis

We are not interested in further information about more than one **DIVERTED airport** so i will drop theses columns furthermore these columns contain almot 100% Nan values beacuse amot all flights didn't need diverted airport or at most only one diverted Airport

In [52]:
Diverted_cols = ['Div2Airport','Div2AirportID','Div2AirportSeqID','Div2WheelsOn','Div2TotalGTime',
 'Div2LongestGTime','Div2WheelsOff','Div2TailNum','Div3Airport','Div3AirportID','Div3AirportSeqID',
 'Div3WheelsOn','Div3TotalGTime','Div3LongestGTime','Div3WheelsOff','Div3TailNum','Div4Airport',
 'Div4AirportID','Div4AirportSeqID','Div4WheelsOn','Div4TotalGTime','Div4LongestGTime','Div4WheelsOff',
 'Div4TailNum','Div5Airport','Div5AirportID','Div5AirportSeqID','Div5WheelsOn','Div5TotalGTime',
 'Div5LongestGTime','Div5WheelsOff','Div5TailNum','Unnamed: 109']

In [53]:
df1=df1.drop(Diverted_cols,axis=1)

In [54]:
df1.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583985 entries, 0 to 583984
Columns: 77 entries, Year to Div1TailNum
dtypes: float64(39), int64(21), object(17)
memory usage: 343.1+ MB


#### Memory usage_2
After removing unnecessary data which describe the Second,third,fourth and fifth diverted airport we decreased memory usage by **26.85%** which becomes **343.1 MB**

#### Redundant columns
the follwing columns are repeted or contain information that not necessary or repetead in another form in another columns so i will drop it for memory issues 

`'DOT_ID_Reporting_Airline'`          didn't provide useful info 

`'IATA_CODE_Reporting_Airline'`       i will use **Reporting_Airline** column instead

`'OriginAirportSeqID'`                i will use **OriginAirportID** as this column is unique for each airport over the years

`'OriginStateFips'`                   this column contain fedral identfication number for each airport 

`'OriginState'`                       **OriginStateName** column in more handy

`'OriginWac'`                         origin area code is not helpful for my analysis

`'DestAirportSeqID'`                  i will use **DestAirportID** as this column is unique for each airport over the years

`'DestState'`                         **DestStateName** column in more handy

`'DestStateFips'`                     this column contain fedral identfication number for each airport 

`'DestWac'`                           Destination area code is not helpful for my analysis

In [55]:
# unrelevant columns

unrelevant_col = ['DOT_ID_Reporting_Airline',
                  'IATA_CODE_Reporting_Airline',
                  'OriginAirportSeqID',
                  'OriginStateFips',
                  'OriginState',
                  'OriginWac',
                  'DestAirportSeqID',
                  'DestState',
                  'DestStateFips',
                  'Div1WheelsOff',                     
                  'Div1TailNum',
                  'Div1AirportSeqID',
                  'DestWac']

In [56]:
df1= df1.drop(unrelevant_col,axis=1)

In [57]:
df1.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583985 entries, 0 to 583984
Columns: 64 entries, Year to Div1LongestGTime
dtypes: float64(37), int64(14), object(13)
memory usage: 285.1+ MB


#### Memory usage_3
After removing Redundant columns we decreased memory usage by **17%** which becomes **285.1 MB**

### Step 2: Datatypes Downcasting

In [58]:
cols=list(df1.select_dtypes('int64').columns)
cols

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginCityMarketID',
 'DestAirportID',
 'DestCityMarketID',
 'CRSDepTime',
 'CRSArrTime',
 'DistanceGroup',
 'DivAirportLandings']

After examining each column and apply pandas.dataframe.describe() method to find statistics about each column its min and max specially to find ranges and which of them fit with **int16** and we find that all of thes columns fit wih **int16** except `'Flight_Number_Reporting_Airline'` but i will convert only **'OriginAirportID','DestAirportID','DistanceGroup'** and the rest will be categorical to save more memory and i will show why the rest are categorical after that

as we know int16 uses 2 bytes of data while **int64** uses 8 bytes of data by this technique we will save alot of memory

In [59]:
int_downcast=['OriginAirportID',
 'DestAirportID',
 'DistanceGroup']


In [60]:
df1[int_downcast]=df1[int_downcast].astype('int16')

In [61]:
df1.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583985 entries, 0 to 583984
Columns: 64 entries, Year to Div1LongestGTime
dtypes: float64(37), int16(3), int64(11), object(13)
memory usage: 275.1+ MB


#### Memory usage_4
After downcasting **int64** columns which can fit with **int16** we decreased memory usage from **285.1 MB** to **275.1 MB**

In [62]:
float_downcast=['Distance',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'DivArrDelay',
 'DivDistance','Div1TotalGTime',
 'Div1LongestGTime',
 'Div1AirportID']

##### First 

`'Flights'` can be converted to int8

##### Second
 **`'Distance','CarrierDelay','WeatherDelay','NASDelay'`,**
 
**`'SecurityDelay','LateAircraftDelay','DivArrDelay','DivDistance'`,**
 
 **`'Div1TotalGTime','Div1LongestGTime'`**
 
**can be converted to float16 as the range for theire values between -32768 and 32767**

other columns like 
**`'DepDel15','ArrDel15','Cancelled','Diverted','DivReachedDest'`** are ust an **indicator** 0,1 so i will convert them into **Boolean** in the next step after converting the previous columns into **float16**

In [63]:
df1['Flights']=df1['Flights'].astype('int8')

df1[float_downcast]=df1[float_downcast].astype('float16')

In [64]:
df1.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583985 entries, 0 to 583984
Columns: 64 entries, Year to Div1LongestGTime
dtypes: float16(11), float64(25), int16(3), int64(11), int8(1), object(13)
memory usage: 234.5+ MB


#### Memory usage_5
After downcasting **float64** columns which can fit with **float16** we decreased memory usage from **275.1 MB**  to **234.5 MB**

In [65]:
bool_downcast=[
 'DepDel15',
 'ArrDel15',
 'Cancelled',
 'Diverted',
 'DivReachedDest']

df1[bool_downcast]=df1[bool_downcast].astype('bool')

In [66]:
df1.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583985 entries, 0 to 583984
Columns: 64 entries, Year to Div1LongestGTime
dtypes: bool(5), float16(11), float64(20), int16(3), int64(11), int8(1), object(13)
memory usage: 215.0+ MB


#### Memory usage_6
After downcasting **some of float64** columns which are just an indicator into **bool** we decreased memory usage from **234.5 MB** to **215 MB**

After that we still have columns their dtypes are **int64** while we can convert them into **int32** to save more memory

In [67]:
int_downcast_32=['Flight_Number_Reporting_Airline',
 'OriginCityMarketID',
 'DestCityMarketID',
 'CRSDepTime',
 'CRSArrTime']


In [68]:
df1[int_downcast_32]=df1[int_downcast_32].astype('int32')

In [69]:
df1.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583985 entries, 0 to 583984
Columns: 64 entries, Year to Div1LongestGTime
dtypes: bool(5), float16(11), float64(20), int16(3), int32(5), int64(6), int8(1), object(13)
memory usage: 203.8+ MB


#### Memory usage_7
After downcasting **'Flight_Number_Reporting_Airline','OriginCityMarketID','DestCityMarketID','CRSDepTime',
 'CRSArrTime'** columns which are **int64** into **int32** we decreased memory usage from **215 MB** to **203.8 MB**


In [70]:
unique_counts = pd.DataFrame.from_records([(col, df1[col].nunique()) for col in df1.columns],
                          columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])

In [71]:
unique_counts.head(20)

,Column_Name,Num_Unique
0,Year,1
2,Month,1
43,Flights,1
1,Quarter,1
39,Diverted,2
34,ArrDel15,2
37,Cancelled,2
23,DepDel15,2
55,DivReachedDest,2
38,CancellationCode,4


### Categorical columns conversion

after investigating the above columns which has less number of unique values i will convert **'DivAirportLandings',
'DepartureDelayGroups','ArrivalDelayGroups','DistanceGroup','Year','Quarter','Month','DayofMonth','DayOfWeek','CancellationCode** and the rest of columns will be converted into appropriate form of date and time

In [72]:
categorical_downcast=['DivAirportLandings',
                      'DepartureDelayGroups',
                      'ArrivalDelayGroups',
                      'DistanceGroup',
                      'Year',
                      'Quarter',
                      'Month',
                      'DayofMonth',
                      'DayOfWeek',
                      'CancellationCode']
df1[categorical_downcast]=df1[categorical_downcast].astype('category')


In [73]:
df1.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583985 entries, 0 to 583984
Columns: 64 entries, Year to Div1LongestGTime
dtypes: bool(5), category(10), float16(11), float64(18), int16(2), int32(5), int8(1), object(12)
memory usage: 168.2+ MB


#### Memory usage_7
After converting **'DepartureDelayGroups','ArrivalDelayGroups','DistanceGroup'** columns which are **float64** into **categorical data-type** we decreased memory usage from **203.8 MB** to **168.2 MB**


#### Time and date columns in float64 Format

we can observe that each one of rest of **float64** columns describe date or time information and after investigating their range all of them can fit into **float16** format this is a transitional step before converting them into approprite **datetime** format

In [74]:
time_float_downcast=['DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'FirstDepTime',
 'TotalAddGTime',
 'LongestAddGTime',
 'DivActualElapsedTime',
 'Div1WheelsOn',
 'CRSArrTime', 'CRSDepTime']

In [75]:
df1[time_float_downcast]=df1[time_float_downcast].astype('float16')

In [76]:
df1.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583985 entries, 0 to 583984
Columns: 64 entries, Year to Div1LongestGTime
dtypes: bool(5), category(10), float16(31), int16(2), int32(3), int8(1), object(12)
memory usage: 105.8+ MB


In [77]:
# hhmm columns in our dataframe
hhmm=['DepTime',
      'CRSArrTime', 
      'CRSDepTime',
      'WheelsOff',
      'WheelsOn',
      'ArrTime']

In [78]:
df1 = df1.dropna(subset= hhmm)

In [79]:
df1.shape

(566924, 64)

In [80]:
df1.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566924 entries, 0 to 583984
Columns: 64 entries, Year to Div1LongestGTime
dtypes: bool(5), category(10), float16(31), int16(2), int32(3), int8(1), object(12)
memory usage: 107.1+ MB


In [81]:
def std_time_1(col):

    Hours = int(col / 100)
    Minute = round(float(col/ 100) - Hours,2)
    Hours = str(int(Hours)).zfill(2)
    Minute = str(Minute).replace('0.','').zfill(2)
    my_str = Hours+':'+Minute
    
    return my_str
    

In [82]:
for i in hhmm:
    df1[i]=df1[i].apply(lambda x:std_time_1(x))

In [83]:
for i in hhmm:
    df1[i]=df1[i].str.replace('24:00','23:59')

In [84]:
def std_time(string):
    return datetime.datetime.strptime(string, '%H:%M').time()

In [85]:
for i in hhmm:
    df1[i]=df1[i].apply(lambda x:std_time(x))

In [86]:
df1[hhmm].head()

,DepTime,CRSArrTime,CRSDepTime,WheelsOff,WheelsOn,ArrTime
0,13:53:00,15:01:00,14:00:00,14:02:00,14:39:00,14:44:00
1,09:03:00,11:18:00,09:35:00,09:57:00,11:13:00,11:19:00
2,06:37:00,08:55:00,06:43:00,06:54:00,08:22:00,08:38:00
3,13:14:00,14:33:00,13:35:00,13:37:00,13:57:00,14:04:00
4,08:26:00,10:04:00,08:36:00,08:52:00,09:59:00,10:09:00


In [87]:
df1[['FlightDate','OriginCityName','OriginStateName','DestCityName','DestStateName']].head()

,FlightDate,OriginCityName,OriginStateName,DestCityName,DestStateName
0,2019-01-04,"San Luis Obispo, CA",California,"San Francisco, CA",California
1,2019-01-04,"Houston, TX",Texas,"Fayetteville, AR",Arkansas
2,2019-01-04,"Springfield, MO",Missouri,"Houston, TX",Texas
3,2019-01-04,"Williston, ND",North Dakota,"Denver, CO",Colorado
4,2019-01-04,"Milwaukee, WI",Wisconsin,"Denver, CO",Colorado


In [88]:
df1['OriginCityName'] = df1['OriginCityName'].str.split(',')
df1['DestCityName'] = df1['DestCityName'].str.split(',')
df1['OriginCityName'] = df1['OriginCityName'].str[0]
df1['DestCityName'] = df1['DestCityName'].str[0]

In [90]:
df1[['FlightDate','OriginCityName','OriginStateName','DestCityName','DestStateName']].head()

,FlightDate,OriginCityName,OriginStateName,DestCityName,DestStateName
0,2019-01-04,San Luis Obispo,California,San Francisco,California
1,2019-01-04,Houston,Texas,Fayetteville,Arkansas
2,2019-01-04,Springfield,Missouri,Houston,Texas
3,2019-01-04,Williston,North Dakota,Denver,Colorado
4,2019-01-04,Milwaukee,Wisconsin,Denver,Colorado


#### Memory usage_8

After converting **'DepTime','CRSArrTime', 'CRSDepTime','WheelsOff','WheelsOn','ArrTime'** columns which are **float16** into **datetime then extract the time part of it** this step increase memory usage from **107.1 MB** To **126.5 MB** but this step is necessary for further analysis 


In [91]:
list(df1.columns)

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginStateName',
 'DestAirportID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestStateName',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'FirstDepTime',
 'TotalAddGTime',
 'LongestAddGTime',
 'DivAirportLandings',
 'DivReachedDest',
 'DivActualElapsedTime',
 'DivArrDelay',
 'DivDistance',
 'Div1Airport',


In [92]:
conn = sqlite3.connect(r'D:\on time data for airlines\2019\2019.db')
c = conn.cursor()

In [94]:
c.execute('''CREATE TABLE IF NOT EXISTS AIR2019_1 
 (Year,
 Quarter,Month,
 DayofMonth,DayOfWeek,
 FlightDate,
 Reporting_Airline,
 Tail_Number,Flight_Number_Reporting_Airline,
 OriginAirportID,
 OriginCityMarketID,
 Origin,
 OriginCityName,
 OriginStateName,
 DestAirportID,
 DestCityMarketID,
 Dest,
 DestCityName,
 DestStateName,
 CRSDepTime,
 DepTime,
 DepDelay,
 DepDelayMinutes,
 DepDel15,
 DepartureDelayGroups,
 DepTimeBlk,
 TaxiOut,
 WheelsOff,
 WheelsOn,
 TaxiIn,
 CRSArrTime,
 ArrTime,
 ArrDelay,
 ArrDelayMinutes,
 ArrDel15,
 ArrivalDelayGroups,
 ArrTimeBlk,
 Cancelled,
 CancellationCode,
 Diverted,
 CRSElapsedTime,
 ActualElapsedTime,
 AirTime,
 Flights,
 Distance,
 DistanceGroup,
 CarrierDelay,
 WeatherDelay,
 NASDelay,
 SecurityDelay,
 LateAircraftDelay,
 FirstDepTime,
 TotalAddGTime,
 LongestAddGTime,
 DivAirportLandings,
 DivReachedDest,
 DivActualElapsedTime,
 DivArrDelay,
 DivDistance,
 Div1Airport,
 Div1AirportID,
 Div1WheelsOn,
 Div1TotalGTime,
 Div1LongestGTime)''')
conn.commit()

In [95]:
df1.to_sql('AIR2019_1', conn, if_exists='replace', index = False)
